In [1]:
import requests
import json
import os
import sys
import pdb

In [2]:
# global names
API_ROUTES = 'https://api-v3.mbta.com/routes'
API_STOPS = 'https://api-v3.mbta.com/stops'
API_LINES = 'https://api-v3.mbta.com/lines'

### Playing around

In [3]:
params = {'filter[type]':0}
response = requests.get(API_ROUTES)
response.json().keys()

dict_keys(['data', 'jsonapi'])

In [4]:
data = response.json()['data']
data[0]

{'attributes': {'color': 'DA291C',
  'description': 'Rapid Transit',
  'direction_destinations': ['Ashmont/Braintree', 'Alewife'],
  'direction_names': ['South', 'North'],
  'fare_class': 'Rapid Transit',
  'long_name': 'Red Line',
  'short_name': '',
  'sort_order': 10010,
  'text_color': 'FFFFFF',
  'type': 1},
 'id': 'Red',
 'links': {'self': '/routes/Red'},
 'relationships': {'line': {'data': {'id': 'line-Red', 'type': 'line'}},
  'route_patterns': {}},
 'type': 'route'}

In [5]:
len(data)

192

## Question 1

Write a program that retrieves data representing all, what we'll call "subway" routes: "Light Rail"
(type 0) and “Heavy Rail” (type 1). The program should list their “long names” on the console.

In [6]:
def get_subway_routes(rail_type):
    
    params = {'filter[type]':rail_type}
    response = requests.get(API_ROUTES,params)
    
    routes = []
    for _,data in enumerate(response.json()['data']):
        routes.append(data['attributes']['long_name'])
    routes = list(set(routes))
    return routes

def print_subway_routes():
    ''' 0: "Light Rail" , 1: “Heavy Rail” '''
    
    lightRoutes = get_subway_routes(0)
    heavyRoutes = get_subway_routes(1)
    print( ', '.join(lightRoutes+heavyRoutes) )

In [7]:
print_subway_routes()

Green Line B, Green Line C, Green Line E, Green Line D, Mattapan Trolley, Blue Line, Red Line, Orange Line


**TODO**: Rather than just saving long_name, save all useful information about routes, create a Route class for that

## Question 2

Extend your program so it displays the following additional information.
1. The name of the subway route with the most stops as well as a count of its stops.
2. The name of the subway route with the fewest stops as well as a count of its stops.
3. A list of the stops that connect two or more subway routes along with the relevant route names for each of those stops.

In [8]:
# the following would not be necessary if Route data stored before
# TODO: use register decorator

def get_subway_route_ids():
    ''' 0: "Light Rail" , 1: “Heavy Rail” '''
    routeIds = []
    for rail_type in range(2):
        params = {'filter[type]':rail_type}
        response = requests.get(API_ROUTES,params)
        for _,data in enumerate(response.json()['data']):
            routeIds.append(data['id'])
    routeIds = list(set(routeIds))
    return routeIds

In [9]:
routeIds = get_subway_route_ids()
len(routeIds)

8

In [10]:
from collections import namedtuple

Stop = namedtuple('Stop','stopId name route')

class Route:
    def __init__(self,routeId,stops):
        self.stops = stops
        self.routeId = routeId

def extract_stops_from_json(jsonResponse):
    stops = []
    for _,jsonStop in enumerate(jsonResponse):
        name = jsonStop['attributes']['name']
        stopId = jsonStop['id']
        # TODO: what if multiple routes associated to same stop?
        # tried, only 1 route appears even if stops connect w
        # other routes
        route = jsonStop['relationships']['route']['data']['id']
        stop = Stop(stopId,name,route)
        stops.append(stop)
    return stops

In [11]:
# generate a list of routes
# each route is just an object with list of stops as attribute

routes = []
allStops = []
for _,routeId in enumerate(routeIds):
    
    # retrieve stops for a particular route
    params = {'filter[route]': routeId, 'include': 'route'}
    response = requests.get(API_STOPS,params)
    stops = extract_stops_from_json(response.json()['data'])
    routes.append(Route(routeId, stops))
    allStops += stops

In [12]:
# print nr. of stops for all routes

for _,route in enumerate(routes):
    print('Route {:10s}: {:2d} stops'.format(route.routeId,len(route.stops)))

Route Green-E   : 20 stops
Route Mattapan  :  8 stops
Route Green-B   : 24 stops
Route Green-D   : 20 stops
Route Red       : 22 stops
Route Blue      : 12 stops
Route Green-C   : 22 stops
Route Orange    : 20 stops


### List stops with multiple routes associated

In [13]:
stopNames = [x.name for x in allStops]
stopIds =  [x.stopId for x in allStops]
stopRoutes = [x.route for x in allStops]

In [14]:
from collections import Counter
stopTuples = Counter(stopNames).most_common()

hubStops = [x for x in stopTuples if x[1] > 1]
print(hubStops)

[('Park Street', 5), ('Copley', 4), ('Arlington', 4), ('Boylston', 4), ('Government Center', 4), ('Haymarket', 3), ('North Station', 3), ('Kenmore', 3), ('Hynes Convention Center', 3), ('Ashmont', 2), ('Saint Paul Street', 2), ('Downtown Crossing', 2), ('State', 2)]


In [15]:
for i,stopTuple in enumerate(hubStops):
    stopName = stopTuple[0]
    idxs = [i for i, x in enumerate(stopNames) if x == stopName]
    routes = [stopRoutes[i] for i in idxs]
    stopId = stopIds[ stopNames.index(stopName) ]
    print('{:10s} {:25s} {}'.format(stopId,stopName,', '.join(routes)))

place-pktrm Park Street               Green-E, Green-B, Green-D, Red, Green-C
place-coecl Copley                    Green-E, Green-B, Green-D, Green-C
place-armnl Arlington                 Green-E, Green-B, Green-D, Green-C
place-boyls Boylston                  Green-E, Green-B, Green-D, Green-C
place-gover Government Center         Green-E, Green-D, Blue, Green-C
place-haecl Haymarket                 Green-E, Green-C, Orange
place-north North Station             Green-E, Green-C, Orange
place-kencl Kenmore                   Green-B, Green-D, Green-C
place-hymnl Hynes Convention Center   Green-B, Green-D, Green-C
place-asmnl Ashmont                   Mattapan, Red
place-stplb Saint Paul Street         Green-B, Green-C
place-dwnxg Downtown Crossing         Red, Orange
place-state State                     Blue, Orange


## Question 3:
Extend your program again such that the user can provide any two stops on the subway routes you listed for question 1.

List a rail route you could travel to get from one stop to the other.

We will not evaluate your solution based upon the efficiency or cleverness of your route-finding solution. Pick a simple solution that answers the question. We will want you to understand and be able to explain how your algorithm performs.
Examples:
1. Davis to Kendall -> Redline
2. Ashmont to Arlington -> Redline, Greenline

In [16]:
class Stop():
    
    def __init__(self):
        return
    
    def get_routes(self):
        return
        
class Route:
    def __init__(self,routeId,stops):
        self.stops = stops
        self.routeId = routeId
        
    def get_connected_routes():
        connectedRoutes = []
        for _,stop in enumerate(self.stops):
            routes = stop.get_routes()
            connectedRoutes.append(routes)
        connectedRoutes = list(set(connectedRoutes))
        return connectedRoutes

class ContainerOfTravels():
    ''' Container with all travel candidates '''
    
    def __init__(self):
        self.travels = []
    
    @property
    def length(self): return len(self.travels)
    
    def pop(self):
        return self.travels.pop(0)
    
    def add(self,travels):
        for travel in travels:
            self.append(travel)

    def update(self,travel):
        ''' update travel candidates by expanding current travel '''
        extendedTravels = travel.extend_travel()
        selected = self.select(extendedTravels)
        for i,travel in enumerate(extendedTravels):
            if selected[i] == True: self.add(travel)
    
    def is_empty(self):
        return (len(self.travels) == 0)
    
    def select(self,travels):
        ''' check which travels should be added to container '''
        selected = []
        allRoutes = self._get_all_routes()
        for travel in travels:
            if (travel.hasLoop() or self._isValid(travel)):
                selected.append(False)
            else:
                selected.append(True)
    
    def _get_all_routes(self):
        ''' which routes are present in container '''
        allRoutes = []
        for travel in self.travels:
            routes = travel.get_routes()
            allRoutes.append(routes)
        return set(allRoutes)
    
    def _isValid(self,travel):
        ''' an invalid travel reaches a route previously
        visited in the past '''
        allRoutes = self._get_all_routes()
        if (travel.get_last() in allRoutes): return False
        else: return True
    
class Travel(object):
    
    def __init__(self):
        self.routes = []
    
    def copy(self):
        travel = Travel()
        travel.routes = self.routes.copy()
        
    def add(self,route):
        self.routes.append(route)
        
    def reaches(destinationRoute):
        try:
            return (self.get_last() == destinationRoute)
        except:
            return False
    
    def get_last(): return self.routes[-1]        
        
    def extend_travel(self):
        
        lastRoute = travel.get_last()
        routes = lastRoute.get_connected_routes()
        
        extendedTravels = []
        for _,route in enumerate(routes):
            travel = self.copy()
            travel.add(route)
            extendedTravels.append(extendedTravels)

        return extendedTravels
        
    def get_routes():
        return
        

MAX_TRAVEL_CANDIDATES = 100

def get_travel(originStop,destinationStop):
    
    originRoutes = originStop.get_routes()
    destinationRoutes = destinationStop.get_routes()
    
    travelCandidates = ContainerOfTravels()
    initTravel = Travel(originRoute)
    travelCandidates.add(initTravel)
    
    while not(travelCandidates.isEmpty()):
        
        travel = travelCandidates.pop()
        
        if travel.reaches(destinationRoute): return travel
        
        travelCandidates.expand(travel) # add more candidates
        
        if travelCandidates.length > MAX_TRAVEL_CANDIDATES:
            raise ValueError('Too many travel candidates')
    
    print('Destination: {} cannot be reached from Origin: {}')
    
    return Travel()